# Mock Data
- Take the Theta file
- predict spec and phot
- distort spec and phot with a gaussian sigma=phot/snr
- snr has to be smaller than 40

In [9]:
import os
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
from astropy.io import fits

from prospect import prospect_args
from prospect.fitting import fit_model
from prospect.io import write_results as writer
from prospect.likelihood import NoiseModel
from prospect.likelihood.kernels import Uncorrelated
from prospect.sources.galaxy_basis import CSPSpecBasis
from prospect.utils.obsutils import fix_obs

import prospect.io.read_results as reader
import toolbox_prospector as toolbox

from sedpy.observate import load_filters

import Build


In [18]:
theta_dic = {}

theta_dic['parnames']        =  ['logzsol', 'dust2', 'logmass', 'logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios', 'dust_index', 'dust1_fraction', 'igm_factor', 'gas_logz', 'gas_logu', 'frac_obrun']
theta_dic['thetas_original'] =  [-0.5,  0.6, 10. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. ,  0. , -2. ,  0. ]
theta_dic['thetas_0']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0. ]
theta_dic['thetas_1']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.1 ]
theta_dic['thetas_2']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.3 ]
theta_dic['thetas_3']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.5 ]
theta_dic['thetas_4']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.7 ]
theta_dic['thetas_5']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.9 ]
theta_dic['thetas_6']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  1. ]

theta_dic['thetas_7']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.15 ]
theta_dic['thetas_8']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.2 ]
theta_dic['thetas_9']        =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.25 ]
theta_dic['thetas_10']       =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.35 ]
theta_dic['thetas_11']       =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.4 ]
theta_dic['thetas_12']       =  [-0.1,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.45 ]

filename = 'mock_thetas.pickle'

# open the file in write mode and save the dictionary as a pickle
with open(path_mock+filename, 'wb') as f:
    pickle.dump(theta_dic, f)

## Read in all the data

In [19]:
path_wdir   =   "/Users/amanda/Desktop/Paper/technical/"
path_data   =   os.path.join(path_wdir, "data/")
path_plots  =   os.path.join(path_wdir, "plots/")
path_output =   os.path.join(path_wdir, "prospector/")
path_flury  =   os.path.join(path_data, 'flury/')
path_mock   =   os.path.join(path_data, 'mock/')

#------------------------------------------------------Emission Lines-----------------------------------------------------------------------

el_table  =  "lzlcs_optlines_obs.csv"
el        =   Table.read(os.path.join(path_flury, el_table), format="ascii")

el.add_column([i for i in range(1,67)], name='id', index=0) #uniform reference for name now possible


#-------------------------------------------------------Photometry--------------------------------------------------------------------------

phot_table  =   'GP_Aperture_Matched_Photometry_v0.fits'
phot_cat    =   fits.open(os.path.join(path_flury, phot_table))
phot        =   phot_cat[1].data

#--------------------------------------------------------Mock Thetas------------------------------------------------------------------------

with open(path_mock+'mock_thetas.pickle', 'rb') as f:
    thetas = pickle.load(f)

#------------------------------------------------------Signal to Noise----------------------------------------------------------------------


snr      = pd.read_csv(path_data + 'snr.csv')
snr_phot = snr['median'][:7] 
snr_el   = snr['median'][7:]

#------------------------------------------------------Mock Photometry----------------------------------------------------------------------

with open(path_data + 'distorted_data.pickle', 'rb') as f:
    dis_data = pickle.load(f)


## Build
- build obs with galaxy 3 because it has all the bands and all the emission lines and it doesnt matter which one
- build model
- build sps

In [20]:
thetas.keys()

dict_keys(['parnames', 'thetas_original', 'thetas_0', 'thetas_1', 'thetas_2', 'thetas_3', 'thetas_4', 'thetas_5', 'thetas_6', 'thetas_7', 'thetas_8', 'thetas_9', 'thetas_10', 'thetas_11', 'thetas_12'])

In [12]:
objid   =  3
obs     =  Build.build_obs(objid=objid)
model   =  Build.build_model(objid=objid, fit_el=True)
sps     =  Build.build_sps()

SFH template = continuity_sfh
redshift init = 0.3061555
FITTING EL MODEL


## Predict 
- spec and phot predicted for the different thetas, which vary frac_obrun from 0, 0.1, 0.3, 0.5, 0.7, 0.9, 1

In [21]:
spec_7, phot_7, _   = model.predict(thetas['thetas_7'], obs=obs, sps=sps)
spec_8, phot_8, _   = model.predict(thetas['thetas_8'], obs=obs, sps=sps)
spec_9, phot_9, _   = model.predict(thetas['thetas_9'], obs=obs, sps=sps)
spec_10, phot_10, _ = model.predict(thetas['thetas_10'], obs=obs, sps=sps)
spec_11, phot_11, _ = model.predict(thetas['thetas_11'], obs=obs, sps=sps)
spec_12, phot_12, _ = model.predict(thetas['thetas_12'], obs=obs, sps=sps)


## Change the SNR 
- the snr can not be larger than 20

In [22]:
snr_el_corrected    = []
snr_phot_corrected  = []

for i in snr_el:
    if i<20:
        snr_el_corrected.append(i)
    else:
        snr_el_corrected.append(20)

for i in snr_phot:
    if i<20:
        snr_phot_corrected.append(i)
    else:
        snr_phot_corrected.append(20)

## Generate the Uncertainties

In [23]:
phot_7_unc  = np.random.normal(loc=0., scale=np.abs(phot_7/snr_phot_corrected))
phot_8_unc  = np.random.normal(loc=0., scale=np.abs(phot_8/snr_phot_corrected))
phot_9_unc  = np.random.normal(loc=0., scale=np.abs(phot_9/snr_phot_corrected))
phot_10_unc = np.random.normal(loc=0., scale=np.abs(phot_10/snr_phot_corrected))
phot_11_unc = np.random.normal(loc=0., scale=np.abs(phot_11/snr_phot_corrected))
phot_12_unc = np.random.normal(loc=0., scale=np.abs(phot_12/snr_phot_corrected))

spec_7_unc  = np.random.normal(loc=0., scale=np.abs(spec_7/snr_el_corrected))
spec_8_unc  = np.random.normal(loc=0., scale=np.abs(spec_8/snr_el_corrected))
spec_9_unc  = np.random.normal(loc=0., scale=np.abs(spec_9/snr_el_corrected))
spec_10_unc = np.random.normal(loc=0., scale=np.abs(spec_10/snr_el_corrected))
spec_11_unc = np.random.normal(loc=0., scale=np.abs(spec_11/snr_el_corrected))
spec_12_unc = np.random.normal(loc=0., scale=np.abs(spec_12/snr_el_corrected))


## Generate the Data
- combine phot with the generated distortion
- take the distortion as the uncertainty
- set the masks to True everywhere
- take the same data than for the galaxies for
    - filters, wave_effective, wavelength, line_ind, cat_row, id, redshift and line_names

In [26]:
#dis_data = {}

index        =  ['7', '8', '9', '10', '11', '12']
phot_all     =  [phot_7, phot_8, phot_9, phot_10, phot_11, phot_12]
phot_all_unc =  [phot_7_unc , phot_8_unc , phot_9_unc , phot_10_unc, phot_11_unc, phot_12_unc]
spec_all     =  [spec_7, spec_8, spec_9, spec_10, spec_11, spec_12]
spec_all_unc =  [spec_7_unc , spec_8_unc , spec_9_unc , spec_10_unc, spec_11_unc, spec_12_unc]

for i, p in zip(index, range(len(phot_all))):
    dis_data['thetas_'+i]      =   thetas['thetas_'+i]
    dis_data['maggies_'+i]     =   phot_all[p] + phot_all_unc[p]
    dis_data['maggies_unc_'+i] =   phot_all_unc[p]
    dis_data['spec_'+i]        =   spec_all[p] + spec_all_unc[p]
    dis_data['spec_unc_'+i]    =   spec_all_unc[p]

#dis_data['filters']         =   obs['filters']
#dis_data['wave_effective']  =   obs['wave_effective']
#dis_data['wavelength']      =   obs['wavelength']
#dis_data['mask']            =   [True for i in dis_data['wavelength']]
#dis_data['phot_mask']       =   [True for i in dis_data['filters']]
#dis_data['line_ind']        =   obs['line_ind']
#dis_data['cat_row']         =   obs['cat_row']
#dis_data['id']              =   obs['id']
#dis_data['z_spec']          =   obs['z_spec']
#dis_data['line_names']      =   obs['line_names']

## Save the Distorted Data in Pickle:

In [27]:
filename = path_data + 'distorted_data.pickle'

with open(filename, 'wb') as f:
    pickle.dump(dis_data, f)


The distorted data is now used as the new obs dictionary in the next step, which is putting everything into prospector again. For that we use DISPRO.py.